In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier

from fluency.preprocessing.prepare_fluency_data import extract_features

## Checking the individual models

In [24]:

def get_predictions(model):
    unconfident_inputs = extract_features("../samples/unconfident.m4a")
    confident_inputs = extract_features("../samples/confident.m4a")
    tim_urban_inputs = extract_features("../samples/tim-urban.m4a")
    almost_silent_inputs = extract_features('../samples/almost silent.m4a')
    stutter_inputs = extract_features("../samples/stutter.m4a")
    
    speeches = ['Unconfident speech', "Confident speech", "Tim urban's speech", "Almost silent speech", "Stuttering speech"]
    classes = ["Low", "Medium", "High"]
    
    inputs = np.array([confident_inputs, unconfident_inputs, tim_urban_inputs, almost_silent_inputs, stutter_inputs])
    
    try:
        res = model.predict_proba(inputs)
    except:
        res = model.predict(inputs)
    
    for i in range(len(res)):
        print(f"{speeches[i]} has fluency prediction {classes[res[i].argmax()]} with max probability {res[i].max()} and all probabilities {res[i]}")
        
    return res

In [25]:
# Models
lgbm = joblib.load("models/weights/lightgbm_model.pkl", "r")
xgbm = joblib.load("models/weights/xgboost_model.pkl", "r")
rf = joblib.load("models/weights/random_forest_model.pkl", "r")
stacking = joblib.load("models/weights/stacking_model.pkl", "r")
voting = joblib.load("models/weights/voting_model.pkl", "r")

In [26]:
print("LGBM:")
lgbm_preds = get_predictions(lgbm)

LGBM:
Unconfident speech has fluency prediction High with max probability 0.9794133759673181 and all probabilities [0.01300782 0.00757881 0.97941338]
Confident speech has fluency prediction High with max probability 0.9880800021804874 and all probabilities [0.007899 0.004021 0.98808 ]
Tim urban's speech has fluency prediction Medium with max probability 0.7397166514997517 and all probabilities [2.59889490e-01 7.39716651e-01 3.93858168e-04]
Almost silent speech has fluency prediction High with max probability 0.9976553342721326 and all probabilities [2.06936549e-03 2.75300240e-04 9.97655334e-01]
Stuttering speech has fluency prediction High with max probability 0.9021741479376008 and all probabilities [0.00285557 0.09497029 0.90217415]


In [27]:
print("XGB:")
xgbm_preds = get_predictions(xgbm)

XGB:
Unconfident speech has fluency prediction High with max probability 0.9801576733589172 and all probabilities [0.00822926 0.01161309 0.9801577 ]
Confident speech has fluency prediction High with max probability 0.9557744860649109 and all probabilities [0.01056902 0.03365654 0.9557745 ]
Tim urban's speech has fluency prediction Medium with max probability 0.5949495434761047 and all probabilities [0.3988108  0.59494954 0.00623962]
Almost silent speech has fluency prediction High with max probability 0.9459777474403381 and all probabilities [0.05056522 0.00345709 0.94597775]
Stuttering speech has fluency prediction High with max probability 0.8401636481285095 and all probabilities [0.03681974 0.12301659 0.84016365]


In [28]:
print("Stack:")
preds = get_predictions(stacking)

Stack:
Unconfident speech has fluency prediction High with max probability 0.940479495076676 and all probabilities [0.02161025 0.03791025 0.9404795 ]
Confident speech has fluency prediction High with max probability 0.7310327682426312 and all probabilities [0.11989947 0.14906777 0.73103277]
Tim urban's speech has fluency prediction Medium with max probability 0.7876480987853998 and all probabilities [0.17179819 0.7876481  0.04055371]
Almost silent speech has fluency prediction High with max probability 0.9327300662612285 and all probabilities [0.02637561 0.04089432 0.93273007]
Stuttering speech has fluency prediction High with max probability 0.6330560648412025 and all probabilities [0.11439143 0.2525525  0.63305606]


In [29]:
print("Vote:")
preds = get_predictions(voting)

Vote:
Unconfident speech has fluency prediction Low with max probability 2 and all probabilities 2
Confident speech has fluency prediction Low with max probability 2 and all probabilities 2
Tim urban's speech has fluency prediction Low with max probability 1 and all probabilities 1
Almost silent speech has fluency prediction Low with max probability 2 and all probabilities 2
Stuttering speech has fluency prediction Low with max probability 2 and all probabilities 2


In [31]:
print("Meta Forest:")
lgbm_preds_inputs = lgbm_preds.argmax(axis=1)
xgbm_preds_inputs = xgbm_preds.argmax(axis=1)

rf_inputs = np.array([[lgbm_pred, xgbm_pred] for lgbm_pred, xgbm_pred in zip(lgbm_preds_inputs, xgbm_preds_inputs)])

res = rf.predict_proba(rf_inputs)

speeches = ['Unconfident speech', "Confident speech", "Tim urban's speech", "Almost silent speech", "Stuttering speech"]
classes = ["Low", "Medium", "High"]
for i in range(len(res)):
    print(f"{speeches[i]} has fluency prediction {classes[res[i].argmax()]} with max probability {res[i].max()} and all probabilities {res[i]}")

Meta Forest:
Unconfident speech has fluency prediction High with max probability 0.900273047316605 and all probabilities [0.03877356 0.06095339 0.90027305]
Confident speech has fluency prediction High with max probability 0.900273047316605 and all probabilities [0.03877356 0.06095339 0.90027305]
Tim urban's speech has fluency prediction Medium with max probability 0.7946079144677753 and all probabilities [0.15347057 0.79460791 0.05192152]
Almost silent speech has fluency prediction High with max probability 0.900273047316605 and all probabilities [0.03877356 0.06095339 0.90027305]
Stuttering speech has fluency prediction High with max probability 0.900273047316605 and all probabilities [0.03877356 0.06095339 0.90027305]
